In [44]:
.libPaths('/disk1/wenqing/anaconda3/envs/wq_R4.2/lib/R/library')

In [1]:
library(pheatmap)
library(DESeq2)
library(vctrs)
library(Seurat)
library(Matrix)
library(patchwork)
library(cowplot)
library(colorspace)
library(edgeR)
library(dplyr)
library(rlang)
library(ggplot2)
library(hdf5r)
library(ggpubr)


Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loa

In [46]:
sysnaptic_genes <- read.table("/disk1/wenqing/tmp_data/ASD/sysnaptic_genes_undergo_editing_in_coding_regions.txt",sep='\n')[,1]

In [ ]:
#gene mat
samples <- c("asd_male_pfc/asd_17/","asd_male_pfc/asd_18/","asd_male_pfc/asd_57/","ctr_male_pfc/ctr_20/","ctr_male_pfc/ctr_32/","ctr_male_pfc/ctr_56/")
celltypes <- c("L2_3_ExN","L4","L5_6","L5_6_CC","IN_VIP","IN-PV","IN-SST","IN-SV2C","Astrocyte","Olig")
Gene_resNumInCelltype <- c()
for(one_celltype in celltypes){
  for(one_sample in samples){

    print(paste0(one_sample," and ",one_celltype," running:"))
    gene_filename=paste0("/disk1/wenqing/tmp_data/ASD/",one_sample,one_celltype,"_geneMat_over1.h5")

    gene_filename
    infile <- hdf5r::H5File$new(filename = gene_filename, mode = 'r') #infile环境类型
    infile[["exprs"]] #环境类型

    counts=infile[["exprs/data"]]
    indices=infile[["exprs/indices"]]
    #indptr:创建矩阵时所需的pointer
    indptr=infile[["exprs/indptr"]]
    shp=infile[["exprs/shape"]]

    barcode <- infile[["barcode/barcode"]][]
    gene_names=infile[["gene/gene"]][]
    #创建矩阵(注意Cell-BLAST的表达矩阵行为细胞，列为基因)
    sparse.mat <- sparseMatrix(
      j = indices[] + 1,
      p = indptr[],
      x = as.numeric(x = counts[]),
      dims = shp[],
      giveCsparse = FALSE
    )

    colnames(x = sparse.mat) <- barcode
    rownames(x = sparse.mat) <- gene_names
    gene_mat <- sparse.mat
    dim(gene_mat)

    rowsum_gene <- rowSums(gene_mat)
     
    for(gene in sysnaptic_genes){
      specificGene_resNumInCelltype <- rowsum_gene[grep(gene,names(rowsum_gene),ignore.case=T)]
      if(length(specificGene_resNumInCelltype)==0){
        specificGene_resNumInCelltype <- 0
      }
      Gene_resNumInCelltype <- c(Gene_resNumInCelltype,specificGene_resNumInCelltype)
    }   
  } 
}
resByCelltype <- as.data.frame(matrix(Gene_resNumInCelltype,nrow=60,byrow=TRUE))
write.csv(resByCelltype,"/disk1/wenqing/tmp_data/ASD/gene_resByCelltype_1.csv",quote=F,row.names=FALSE,col.names=FALSE)


In [115]:
samples <- c("asd_17_","asd_18_","asd_57_","ctr_20_","ctr_32_","ctr_56_")
celltypes <- c("L2_3_ExN","L4","L5_6","L5_6_CC","IN_VIP","IN_PV","IN_SST","IN_SV2C","Astrocyte","Olig")
all_rownames <- c()
for(one_celltype in celltypes){
  for(one_sample in samples){
    rowname <- paste0(one_sample,one_celltype)
    all_rownames <- c(all_rownames,rowname)
  }
}


In [159]:
rownames(resByCelltype) <- all_rownames
colnames(resByCelltype) <- sysnaptic_genes
write.csv(resByCelltype,"/disk1/wenqing/tmp_data/ASD/gene_resByCelltype_1.csv",quote=F,row.names=TRUE,col.names=FALSE)


Warning message in write.csv(resByCelltype, "/disk1/wenqing/tmp_data/ASD/gene_resByCelltype_1.csv", :
“attempt to set 'col.names' ignored”


In [163]:
#####各细胞类型突触相关基因在res数量上的差异（asd Vs ctr）
data <- read.csv("/disk1/wenqing/tmp_data/ASD/gene_resByCelltype_1.csv")
head(data)
rownames(data) <- data[,1]
data <- data[,-1]
head(data)
data <- t(data)
head(data)

,X,CYFIP2,FLNA,GABRA3,GRIA2,GRIA3,GRIA4,GRIK1,GRIK2,HTR2C,KCNA1
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,asd_17_L2_3_ExN,14,0,3,2,15,7,407,83,0,0
2,asd_18_L2_3_ExN,1,0,5,8,32,8,10,2,0,0
3,asd_57_L2_3_ExN,328,0,89,17,3,59,4,29,0,3
4,ctr_20_L2_3_ExN,6,0,13,142,2,3,34,13,0,13
5,ctr_32_L2_3_ExN,15,0,16,1,8,18,20,35,0,12
6,ctr_56_L2_3_ExN,4,0,9,9,12,3,0,4,0,21


,CYFIP2,FLNA,GABRA3,GRIA2,GRIA3,GRIA4,GRIK1,GRIK2,HTR2C,KCNA1
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
asd_17_L2_3_ExN,14,0,3,2,15,7,407,83,0,0
asd_18_L2_3_ExN,1,0,5,8,32,8,10,2,0,0
asd_57_L2_3_ExN,328,0,89,17,3,59,4,29,0,3
ctr_20_L2_3_ExN,6,0,13,142,2,3,34,13,0,13
ctr_32_L2_3_ExN,15,0,16,1,8,18,20,35,0,12
ctr_56_L2_3_ExN,4,0,9,9,12,3,0,4,0,21


,asd_17_L2_3_ExN,asd_18_L2_3_ExN,asd_57_L2_3_ExN,ctr_20_L2_3_ExN,ctr_32_L2_3_ExN,ctr_56_L2_3_ExN,asd_17_L4,asd_18_L4,asd_57_L4,ctr_20_L4,⋯,asd_57_Astrocyte,ctr_20_Astrocyte,ctr_32_Astrocyte,ctr_56_Astrocyte,asd_17_Olig,asd_18_Olig,asd_57_Olig,ctr_20_Olig,ctr_32_Olig,ctr_56_Olig
CYFIP2,14,1,328,6,15,4,13,1,0,1,⋯,0,0,0,0,0,1,0,10,8,1
FLNA,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
GABRA3,3,5,89,13,16,9,37,3,26,15,⋯,0,0,0,0,0,3,0,1,0,0
GRIA2,2,8,17,142,1,9,3,4,4,6,⋯,0,0,5,3,1,1,0,4,3,5
GRIA3,15,32,3,2,8,12,8,34,0,8,⋯,0,0,0,0,5,4,0,2,2,14
GRIA4,7,8,59,3,18,3,21,50,6,32,⋯,1,9,0,0,1,2,3,3,8,22


In [164]:
celltypes <- c("L2_3_ExN","L4","L5_6","L5_6_CC","IN_VIP","IN_PV","IN_SST","IN_SV2C","Astrocyte","Olig")
#celltypes <- c("IN_SV2C","Astrocyte","Olig")
i=1
j=6
for(one_celltype in celltypes){
    range <- seq(i,j,by=1)
    sub_data <- data[,range]
    annotation_col = data.frame(
        condition = factor(rep(c("Asd","Ctr"),each=3)),
        phases = factor(c("17","18","57","20","23","56"))
        #phases = factor(rep(c("16","20",'25'),times=3))
    )
    rownames(annotation_col) = colnames(sub_data)
    head(annotation_col)


    #差异分析
    dds <- DESeqDataSetFromMatrix(countData = sub_data, colData = annotation_col, design = ~ condition)


    dds <- DESeq(dds)
    res <- results(dds,contrast = c("condition","Asd","Ctr"))
    res1 <- data.frame(res, stringsAsFactors = FALSE, check.names = FALSE)
    write.table(res1, paste0("/disk1/wenqing/tmp_data/ASD/sysnaptic_genes_De/",one_celltype,"_De.txt"), col.names = NA, sep = '\t', quote = FALSE)  
    i <- i+6
    j <- j+6  

}


estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

Warning message in lfproc(x, y, weights = weights, cens = cens, base = base, geth = geth, :
“Estimated rdf < 1.0; not estimating variance”
final dispersion estimates

fitting model and testing

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specif

ERROR: Error in estimateSizeFactorsForMatrix(counts(object), locfunc = locfunc, : every gene contains at least one zero, cannot compute log geometric means


In [153]:
head(sub_data)

,asd_17_IN_PV,asd_18_IN_PV,asd_57_IN_PV,ctr_20_IN_PV,ctr_32_IN_PV,ctr_56_IN_PV,asd_17_IN_SST
CYFIP2,2,1,5,1,4,6,0
FLNA,0,0,0,0,0,0,0
GABRA3,0,1,0,0,0,0,0
GRIA2,4,5,7,2,2,10,1
GRIA3,0,3,0,0,2,2,0
GRIA4,4,21,2,6,2,19,0


In [146]:

    #差异分析
    dds <- DESeqDataSetFromMatrix(countData = sub_data, colData = annotation_col, design = ~ condition)



class: DESeqDataSet 
dim: 10 6 
metadata(1): version
assays(1): counts
rownames(10): CYFIP2 FLNA ... HTR2C KCNA1
rowData names(0):
colnames(6): asd_17_IN_VIP asd_18_IN_VIP ... ctr_32_IN_VIP
  ctr_56_IN_VIP
colData names(2): condition phases

In [147]:

    dds <- DESeq(dds)


estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

Warning message in lfproc(x, y, weights = weights, cens = cens, base = base, geth = geth, :
“procv: no points with non-zero weight”
Warning message in lfproc(x, y, weights = weights, cens = cens, base = base, geth = geth, :
“procv: no points with non-zero weight”
Warning message in lfproc(x, y, weights = weights, cens = cens, base = base, geth = geth, :
“procv: no points with non-zero weight”
Warning message in lfproc(x, y, weights = weights, cens = cens, base = base, geth = geth, :
“procv: no points with non-zero weight”
Warning message in lfproc(x, y, weights = weights, cens = cens, base = base, geth = geth, :
“procv: no points with non-ze

ERROR: Error in lfproc(x, y, weights = weights, cens = cens, base = base, geth = geth, : newsplit: out of vertex space


In [ ]:
    res <- results(dds,contrast = c("condition","Asd","Ctr"))
    res1 <- data.frame(res, stringsAsFactors = FALSE, check.names = FALSE)

In [ ]:
    res <- results(dds,contrast = c("condition","Asd","Ctr"))
    res1 <- data.frame(res, stringsAsFactors = FALSE, check.names = FALSE)

In [127]:
sub_data <- data[,range]
annotation_col = data.frame(
  condition = factor(rep(c("Asd","Ctr"),each=3)),
  phases = factor(c("17","18","57","20","23","56"))
  #phases = factor(rep(c("16","20",'25'),times=3))
)
rownames(annotation_col) = colnames(sub_data)
head(annotation_col)


#差异分析
dds <- DESeqDataSetFromMatrix(countData = sub_data, colData = annotation_col, design = ~ condition)


dds <- DESeq(dds)
res <- results(dds,contrast = c("condition","Asd","Ctr"))
res1 <- data.frame(res, stringsAsFactors = FALSE, check.names = FALSE)
write.table(res1, "/disk1/wenqing/tmp_data/ASD/sysnaptic_genes_De/L2_3_ExN_De.txt", col.names = NA, sep = '\t', quote = FALSE)
#res <- subset(res, padj < 0.1 & abs(log2FoldChange) > 1)



,condition,phases
,<fct>,<fct>
asd_17_L2_3_ExN,Asd,17
asd_18_L2_3_ExN,Asd,18
asd_57_L2_3_ExN,Asd,57
ctr_20_L2_3_ExN,Ctr,20
ctr_32_L2_3_ExN,Ctr,23
ctr_56_L2_3_ExN,Ctr,56


In [ ]:

#filtering
res1 <- res1[order(res1$padj, res1$log2FoldChange, decreasing = c(FALSE, TRUE)), ]

#log2FC≥1 & padj<0.1 标识 up，代表显著上调的基因
#log2FC≤-1 & padj<0.1 标识 down，代表显著下调的基因
#其余标识 none，代表非差异的基因
res1[which(res1$log2FoldChange >= 1 & res1$padj < 0.1),'sig'] <- 'up'
res1[which(res1$log2FoldChange <= -1 & res1$padj < 0.1),'sig'] <- 'down'
res1[which(abs(res1$log2FoldChange) <= 1 | res1$padj >= 0.1),'sig'] <- 'none'

#输出选择的差异基因总表
res1_select <- subset(res1, sig %in% c('up', 'down'))
#write.table(res1_select, file = './analysis/ASD/deseq_asd_ctr_select.txt', sep = '\t', col.names = NA, quote = FALSE)

#根据 up 和 down 分开输出
res1_up <- subset(res1, sig == 'up')
res1_down <- subset(res1, sig == 'down')

#write.table(res1_up, file = 'control_treat.DESeq2.up.txt', sep = '\t', col.names = NA, quote = FALSE)
#write.table(res1_down, file = 'control_treat.DESeq2.down.txt', sep = '\t', col.names = NA, quote = FALSE)

#pdf("/disk1/wenqing/tmp_data/ASD/sysnaptic_genes_De_L2_3.pdf")
pheatmap(sub_data, annotation_col = annotation_col, annotation_legend = TRUE,cluster_cols = F,cluster_rows = T,show_colnames=F)
#dev.off()

In [132]:
library(ggpubr)
library(ggsignif)
library(ggplot2)
library(colorspace)
colors<-qualitative_hcl(10,palette = "Set3")


In [ ]:

compaired <- list(c("Microglia","CGE-derived InN"), 
                  c("Microglia","MGE-derived InN"), 
                  c("Microglia","Olig"),
                  c("Microglia","Astrocyte"),
                  c("Microglia","DG ExN"),
                  c("Microglia","Non_DG ExN"),
                  c("CGE-derived InN","MGE-derived InN"),
                  c("CGE-derived InN","Olig"),
                  c("CGE-derived InN","Astrocyte"),
                  c("CGE-derived InN","DG ExN"),
                  c("CGE-derived InN","Non_DG ExN"),
                  c("MGE-derived InN","Olig"),
                  c("MGE-derived InN","Astrocyte"),
                  c("MGE-derived InN","DG ExN"),
                  c("MGE-derived InN","Non_DG ExN"),
                  c("Olig","Astrocyte"),
                  c("Olig","DG ExN"),
                  c("Olig","Non_DG ExN"),
                  c("Astrocyte","DG ExN"),
                  c("DG ExN","Non_DG ExN")
                  )


In [ ]:
data_res$group = factor(data_res$group, levels=c("Microglia","CGE-derived InN","MGE-derived InN","Olig","Astrocyte","DG ExN","Non_DG ExN"))
pdf("./finalVer_res/GW18/ResByCellType_ReadOver.pdf")
ggplot(data_res, aes(group, RES, fill = group),group = factor(1)) +
  geom_boxplot() +
  #ylim(1.5, 6.5) +
  # geom_signif(comparisons = compaired,
  #   step_increase = 0.05,
  #   map_signif_level = T,
  #   textsize = 2.3,
  #   test = wilcox.test)+
  theme(axis.title.x = NULL, axis.text.x = element_text(size = rel(0.85)),legend.position="none")+
  scale_fill_manual(values=colors)+
dev.off()

compaired_1 <- list(
                  c("Microglia","MGE-derived InN"), 
                  c("Microglia","Olig"),
                  c("Microglia","Astrocyte"),
                  c("Microglia","DG ExN"),
                  c("Microglia","Non_DG ExN"),
                  c("CGE-derived InN","MGE-derived InN"),
                  c("CGE-derived InN","Olig"),
                  c("CGE-derived InN","Astrocyte"),
                  c("CGE-derived InN","DG ExN"),
                  c("CGE-derived InN","Non_DG ExN"),
                  c("MGE-derived InN","Olig"),
                  c("MGE-derived InN","Astrocyte"),
                  c("MGE-derived InN","DG ExN"),
                  c("MGE-derived InN","Non_DG ExN"),
                  c("Olig","Non_DG ExN"),
                  c("Astrocyte","DG ExN"),
                  c("DG ExN","Non_DG ExN")
)

data_res_deGeneNum$group = factor(data_res_deGeneNum$group, levels=c("Microglia","CGE-derived InN","MGE-derived InN","Olig","Astrocyte","DG ExN","Non_DG ExN"))
pdf("~/SPRINT/bulk/18/filteredRES/resOnGenes_over1de11/uniq_res/ResBygenes_filterRes1De11.pdf")
ggplot(data_res_deGeneNum, aes(group, RES, fill = group),group = factor(1)) +
  geom_boxplot() +
  #ylim(0, 200) +
  geom_signif(comparisons = compaired_1,
              step_increase = 0.05,
              map_signif_level = T,
              textsize = 2.3,
              test = wilcox.test)+
  theme(axis.title.x = NULL, axis.text.x = element_text(size = rel(0.85)),legend.position="none")+
  scale_fill_manual(values=colors)
dev.off()

In [1]:
876/192634
7269/679029

[1] 0.004547484

[1] 0.01070499